<a href="https://colab.research.google.com/github/jpastolfo/Seismic-Velocity-Model-Inversion/blob/main/modelos(versao-1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos UNet e UNet_mod e a função loss
###### No treinamento, chamar **from modelos import get_unet, get_unet_mod, ssim_loss, ssim_loss_rsme**

In [ ]:
#### DEEP LEARNING IMPORTS
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D, UpSampling2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tensorflow.keras import backend as K

In [ ]:
def conv2d_block(input_tensor, n_filters, momentum=0.9, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization(momentum=momentum)(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [ ]:
# defenição das arquiteturas  
def get_unet(input_img, output_filters=1, n_filters = 16, dropout = 0.2, moment=0.9, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, momentum=moment, kernel_size = 6, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 6, momentum=moment, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 6, momentum=moment , batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 6, momentum=moment , batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 6, momentum=moment , batchnorm = batchnorm)

    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, momentum=moment ,batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, momentum=moment , batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, momentum=moment , batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, momentum=moment , batchnorm = batchnorm)

    outputs = Conv2D(output_filters, (1, 1), activation='linear')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model
    

In [ ]:
def get_unet_mod(input_img, output_filters=1, n_filters = 16, dropout = 0.2, moment=0.9, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, momentum=moment, kernel_size = 6, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 6, momentum=moment, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 6, momentum=moment , batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 6, momentum=moment , batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 6, momentum=moment , batchnorm = batchnorm)

    # Expansive Path
    u6 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, momentum=moment ,batchnorm = batchnorm)
 

    u7 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, momentum=moment , batchnorm = batchnorm)

    u8 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, momentum=moment , batchnorm = batchnorm)


    u9 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest')(c8)

    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, momentum=moment , batchnorm = batchnorm)

    outputs = Conv2D(output_filters, (1, 1), activation='linear')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model


In [ ]:
# Loss functtion similaridade estrutural
coef=0.5
def ssim_loss(y_true, y_pred):
      return 1-tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.5,filter_size=5))
def ssim_loss_rsme(y_true, y_pred):
    ssim2=1-tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.5,filter_size=3))
    l2=K.sum(((y_true-y_pred)**2), axis=-1)
    loss=(coef*ssim2)+(l2*(1-coef))
    return loss